In [7]:
import pandas as pd
from sqlalchemy import create_engine
from langchain_core.prompts import PromptTemplate
from langchain_ollama import ChatOllama   # ✅ updated import
from datetime import datetime

# -------------------------------
# Database connection parameters
# -------------------------------
db_params = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "pgpw",
    "host": "localhost",
    "port": "5432"
}

# Create the SQLAlchemy engine
engine = create_engine(
    f'postgresql://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}'
)

# -------------------------------
# Query data
# -------------------------------
query = """
SELECT research_id, age, sex, race_level_1, race_level_2, ethnicity, zip_code, county,
       adi_state, adi_national, year, asthma, diabetes, diabetes_poor_control,
       diabetes_type, hba1c_result, hypertension, bp_control, bp_result,
       obesity, prediabetes, pd_type, tobacco, bcs, ccs, col
FROM public.wny_health_v1;
"""

df = pd.read_sql(query, engine)

# -------------------------------
# Clean and summarize conditions
# -------------------------------
condition_columns = [
    'asthma', 'diabetes', 'diabetes_poor_control', 'hypertension',
    'obesity', 'prediabetes', 'tobacco', 'bcs', 'ccs', 'col'
]

# Standardize values to uppercase strings
df[condition_columns] = df[condition_columns].apply(
    lambda col: col.astype(str).str.strip().str.upper()
)

# Consider YES/Y/1/TRUE/T as positive
positive_mask = df[condition_columns].isin(['YES', 'Y', '1', 'TRUE', 'T'])
summary_counts = positive_mask.sum().astype(int)

# Format for report
summary_df = summary_counts.reset_index()
summary_df.columns = ['Condition', 'Count']
summary_text = summary_df.to_string(index=False)

# -------------------------------
# Prompt template
# -------------------------------
today = datetime.today().strftime('%Y-%m-%d')

template = """
You are a public health epidemiologist.

Today's date is {today}.

Here is a summary of chronic condition counts in Western New York based on current health data:

{summary_text}

Write a clear and professional epidemiology report that:
- Highlights key prevalence patterns
- Identifies any major public health concerns
- Provides actionable insights or recommendations
"""

prompt = PromptTemplate(
    input_variables=["today", "summary_text"],
    template=template
)

# -------------------------------
# Runnable pipeline (Prompt → LLM)
# -------------------------------
llm = ChatOllama(model="llama3.2", temperature=0.3)

# Compose pipeline
chain = prompt | llm

# Run with inputs
report = chain.invoke({"today": today, "summary_text": summary_text})

# -------------------------------
# Output
# -------------------------------
print("\n🧾 Generated Epidemiology Report:\n")
print(report.content)


🧾 Generated Epidemiology Report:

**Epidemiological Report: Chronic Conditions in Western New York (2025)**

**Executive Summary**

This report presents an overview of chronic condition prevalence in Western New York, based on current health data. The analysis highlights key patterns and trends, identifies major public health concerns, and provides actionable insights for policymakers, healthcare providers, and community stakeholders.

**Prevalence Patterns**

Our analysis reveals the following key patterns:

1. **Obesity and Hypertension**: Obesity (264 cases) and hypertension (228 cases) are the most prevalent chronic conditions in Western New York, accounting for approximately 40% of all chronic conditions.
2. **Tobacco-Related Conditions**: Tobacco use is a significant risk factor for various chronic conditions, including asthma (77 cases), diabetes (98 cases), and tobacco-related cancers (37 BCS, 31 CCS, and 63 Col).
3. **Diabetes and Poor Control**: Diabetes (98 cases) and poor 

# CDC API

In [6]:
import pandas as pd
import requests
from sqlalchemy import create_engine
from langchain_core.prompts import PromptTemplate
from langchain_ollama import ChatOllama
from datetime import datetime

# ────────────── 1. DATABASE CONNECTION ──────────────
db_params = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "pgpw",
    "host": "localhost",
    "port": "5432"
}

engine = create_engine(
    f'postgresql://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}'
)

# ────────────── 2. QUERY LOCAL HEALTH DATA ──────────────
query = """
SELECT research_id, age, sex, race_level_1, race_level_2, ethnicity, zip_code, county,
       adi_state, adi_national, year, asthma, diabetes, diabetes_poor_control,
       diabetes_type, hba1c_result, hypertension, bp_control, bp_result,
       obesity, prediabetes, pd_type, tobacco, bcs, ccs, col
FROM public.wny_health_v1;
"""

df = pd.read_sql(query, engine)

# ────────────── 3. SUMMARIZE LOCAL CONDITIONS ──────────────
condition_columns = [
    'asthma', 'diabetes', 'diabetes_poor_control', 'hypertension',
    'obesity', 'prediabetes', 'tobacco', 'bcs', 'ccs', 'col'
]

df[condition_columns] = df[condition_columns].apply(
    lambda col: col.astype(str).str.strip().str.upper()
)

positive_mask = df[condition_columns].isin(['YES', 'Y', '1', 'TRUE', 'T'])
summary_counts = positive_mask.sum().astype(int)

summary_df = summary_counts.reset_index()
summary_df.columns = ['Condition', 'Count']
summary_text = summary_df.to_string(index=False)

# ────────────── 4. CDC API COMPARISON ──────────────
cdc_endpoint = "https://data.cdc.gov/resource/uh6h-vyht.json"

conditions_to_fetch = {
    "asthma": "Current asthma prevalence among adults aged >= 18 years",
    "diabetes": "Diagnosed diabetes among adults aged >= 18 years",
    "hypertension": "Self-reported high blood pressure among adults aged >= 18 years",
    "obesity": "Obesity among adults aged >= 18 years"
}

cdc_stats = {}

for key, indicator in conditions_to_fetch.items():
    params = {
        "$limit": 1,
        "locationabbr": "NY",
        "question": indicator,
        "datavalueunit": "%",
        "stratificationcategory1": "Overall",
        "$order": "year DESC"
    }
    response = requests.get(cdc_endpoint, params=params)
    if response.ok and response.json():
        entry = response.json()[0]
        cdc_stats[key] = {
            "percent": entry.get("datavalue"),
            "year": entry.get("year")
        }
    else:
        cdc_stats[key] = {"percent": "N/A", "year": "N/A"}

# Format CDC Summary
cdc_summary = "\nCDC Comparison Data (New York):\n"
for condition in conditions_to_fetch:
    percent = cdc_stats[condition]["percent"]
    year = cdc_stats[condition]["year"]
    cdc_summary += f"- {condition.capitalize()}: {percent}% in {year}\n"

# ────────────── 5. FORMAT PROMPT ──────────────
today = datetime.today().strftime('%Y-%m-%d')

template = """
You are a public health epidemiologist.

Today's date is {today}.

Here is a summary of chronic condition counts in Western New York based on current health data:

{summary_text}

Here is the most recent CDC-reported prevalence data for New York:

{cdc_summary}

Write a clear and professional epidemiology report that:
- Highlights key prevalence patterns
- Identifies discrepancies between local and CDC data
- Discusses possible reasons for differences
- Provides actionable insights or recommendations
"""

prompt = PromptTemplate(
    input_variables=["today", "summary_text", "cdc_summary"],
    template=template
)

# ────────────── 6. GENERATE REPORT (Runnable Pipeline) ──────────────
llm = ChatOllama(model="llama3.2", temperature=0.3)

# Compose pipeline
chain = prompt | llm

# Run with inputs
report = chain.invoke({
    "today": today,
    "summary_text": summary_text,
    "cdc_summary": cdc_summary
})

# ────────────── 7. OUTPUT REPORT ──────────────
print("\n🧾 Generated Epidemiology Report:\n")
print(report.content)


🧾 Generated Epidemiology Report:

**Epidemiological Report: Chronic Condition Prevalence Patterns in Western New York**

**Date:** December 17, 2025

**Introduction:**
As a public health epidemiologist, it is essential to analyze the chronic condition prevalence data in Western New York to identify key patterns, discrepancies with CDC-reported data, and potential reasons for these differences. This report aims to provide actionable insights and recommendations to inform public health strategies.

**Key Prevalence Patterns:**

1. **Obesity:** The highest prevalence of obesity (264 cases) is observed in Western New York, which is significantly higher than the CDC-reported prevalence rate.
2. **Hypertension:** Hypertension is the second most prevalent chronic condition, with 228 cases, also exceeding the CDC-reported rates.

**Discrepancies between Local and CDC Data:**

1. **Asthma:** The local asthma count (77 cases) does not match the CDC-reported prevalence rate.
2. **Diabetes:** Sim